In [ ]:
import cv2 
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("kaggle.csv")
df_old = pd.read_csv("kaggle_old.csv")

In [ ]:
df["old"]=df_old["Category"]

In [ ]:
df_old

In [ ]:
df[df["Category"]!=df["old"]]

In [ ]:
other_ids = df[df["Category"]==df["old"]]["Id"].to_list()

In [ ]:
fix, axes = plt.subplots(nrows=len(other_ids), ncols=2, figsize=(10, 500))
for i, id in enumerate(other_ids):
    path = Path(f"data/test_images/mistery_category/{id}.jpg")
    crop_path = Path(f"data_crop/test_images/mistery_category/{id}.jpg")
    img = cv2.imread(str(path))
    img_crop = cv2.imread(str(crop_path))
    axes[i, 0].imshow(img)
    axes[i, 1].imshow(img_crop)
plt.show()


In [ ]:

fix, axes = plt.subplots(1, 2)
img = cv2.imread(str(path))
img_crop = cv2.imread(str(crop_path))
axes[0].imshow(img)
axes[1].imshow(img_crop)



In [ ]:
import shutil
from pathlib import Path

import cv2
import pandas as pd
import PIL
from tqdm import trange
import torch
import numpy as np
import sys
sys.path.append("detectron2/detectron2")

import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

setup_logger()
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cpu"

detectron = DefaultPredictor(cfg)

def detect(img):
    width, height, _ = img.shape

    outputs = detectron(img)

    classes = outputs["instances"].get_fields()["pred_classes"]
    bird_indices = np.where(classes == 14)[0]
    if len(bird_indices) == 0:
        left, top, right, bottom = 0, 0, width, height
    else:
        scores = outputs["instances"].get_fields()["scores"]
        for j in range(len(scores)):
            if j not in bird_indices:
                scores[j] = 0
        argmax = scores.argmax().item()
        box = outputs["instances"].get_fields()["pred_boxes"].tensor[argmax]
        left, top, right, bottom = box
        # left -= 10
        # right += 10
        # top -= 10
        # bottom += 10
        left, top, right, bottom = list(map(lambda x: int(x), (left, top, right, bottom)))
        
    return (left, top, right, bottom)

def crop(img, bbox, augmentation_factor = 0.05):
    rows, columns, channels = img.shape
    
    left, top, right, bottom = bbox
    
    center_vertical = (top+bottom)/2
    center_horizontal = (left+right)/2
    
    max_edge = max(bottom-top, right-left)
    size = (1 + augmentation_factor) * max_edge
    
    new_top, new_bottom = center_vertical - size/2, center_vertical + size/2
    new_left, new_right = center_horizontal - size/2, center_horizontal + size/2
    
    center_vertical = center_vertical - min(new_top, 0) + min(rows - new_bottom, 0)
    center_horizontal = center_vertical - min(new_left, 0) + min(rows - new_right, 0)
    
    new_top, new_bottom = int(center_vertical - size/2), int(center_vertical + size/2)
    new_left, new_right = int(center_horizontal - size/2), int(center_horizontal + size/2)
    
    img_crop = img[new_top:new_bottom, new_left:new_right]
    return img_crop

def extract_one(src_path, dst_path):
    img = cv2.imread(src_path)
    bbox = detect(img)
    img_crop = crop(img, bbox)
    cv2.imwrite(dst_path, img_crop)

def detect_all(src_folder):
    files = list(src_folder.rglob("*.jpg"))
    for i in trange(len(files)):
        f = files[i]
        try:
            extract_one(str(f), str(f))
        except:
            print(f"Error with img: {f} !")

In [ ]:
src_folder = Path("data_new")
f = list(src_folder.rglob("*.jpg"))[0]
extract_one(str(f), str(f))


In [ ]:
list(src_folder.rglob("*.jpg"))[0]

In [ ]:
path = Path("../data/test_images")
len(list(path.rglob("*.jpg")))

In [ ]:
exp_files = {
    "Inception":"inception_2022-11-27-00-54", 
    "VGG": "vgg_2022-11-27-00-55", 
    "Resnet":"resnet_2022-11-27-00-56",
    "ViT": "ViT-crops_2022-11-28-00-59",
}

In [ ]:
train_df = pd.DataFrame(columns = exp_files.keys())
for name, exp in exp_files.items():
    path = Path(f"EXPERIMENTS/{exp}/metrics/learning_curves.csv")
    train_df[name] = pd.read_csv(path, index_col=0)["train_loss"]
train_df = train_df.fillna(method="ffill")

test_df = pd.DataFrame(columns = exp_files.keys())
for name, exp in exp_files.items():
    path = Path(f"EXPERIMENTS/{exp}/metrics/learning_curves.csv")
    test_df[name] = pd.read_csv(path, index_col=0)["val_loss"]
test_df = test_df.fillna(method="ffill")

In [ ]:
import seaborn as sns
sns.set_theme('notebook')

In [ ]:
train_df.plot()
plt.xlabel("epochs", fontsize=15)
plt.ylabel("loss", fontsize=15)


In [ ]:
test_df.plot()
plt.xlabel("epochs", fontsize=15)
plt.ylabel("loss", fontsize=15)

In [ ]:
test_df.plot()
plt.title("Accuracy at each epoch - Validation set")